In [15]:

# the follwing script does the dimension analysis/calculation
import numpy as np
import re
# input is of the format
# MPa h d-2, kg m-3, or 
#Pa => N/m2 => kg m s-2 m-2=[kg m-1 s-2]
class U:
    def __init__(self,ustr=None,scal=None):
        self.vvec = np.zeros(6)
        self.scal =1.0
        if scal:
            self.scal=scal
        if ustr:                    
            """
            parse input unit_str into a string vector
            unit_str is case sensitive 
            """
            svec=ustr.split()
            #standarize the array into a vector by the order
            # mass mol time length temperature pressure
            # kilogram: kg, mol: mole, time:s, meter:m, kelvin:K, Pascal:Pa
            id_mass=1;id_mol=2;id_time=3;id_len=4;id_temp=5;id_pres=0
            
            for s in svec:
                match = re.search(r'-?\d+', s)
                if match:
                    val=match.group()
                    ss=s[:match.start()]
                else:
                    val=1
                    ss=s
                
                if ss=='kg' or ss=='g' or ss=='mg' or ss=='t':
                    ids=id_mass
                    if ss=='g':
                        #gram
                        self.scal=self.scal*1.e-3
                    elif ss=='mg':
                        #milligram
                        self.scal=self.scal*1.e-6
                    elif ss=='t':
                        #ton
                        self.scal=self.scal*1.e3
                    ss='kg'    
                elif ss == 'mol' or ss=='mmol' or ss=='umol':
                    ids=id_mol
                    if ss=='mmol':
                        #millimole
                        self.scal=self.scal*1.e-3
                    elif ss=='umol':
                        #micromole
                        self.scal=self.scal*1.e-6
                    ss='mol'    
                elif ss=='s' or ss=='h':
                    ids=id_time
                    if ss=='s':
                        #second'
                        self.scal=self.scal/3600.
                    ss='h'    
                elif ss=='m' or ss=='mm' or ss=='cm' or ss=='km':                    
                    ids=id_len
                    if ss=='mm':
                        self.scal=self.scal*1.e-3
                    elif ss=='cm':
                        self.scal=self.scal*1.e-2
                    elif ss=='km':
                        self.scal=self.scal*1.e3
                    ss='m'
                elif ss=='K':
                    ids=ids_temp                                
                elif ss=='Pa' or ss=='MPa' or ss=='kPa':
                    ids=id_pres 
                    if ss=='MPa':
                        self.scal=self.scal*1.e3
                    elif ss=='Pa':
                        self.scal=self.scal=1.e-3
                    ss='kPa'    
                self.vvec[ids]=val
        # Override the multiplication operator (*)
    def __mul__(self, other):
        if isinstance(other, U):
            res=U()
            res.scal=self.scal*other.scal
            for ii in range(6):
                res.vvec[ii]=self.vvec[ii]+other.vvec[ii]
            return res

    def __truediv__(self, other):
        if isinstance(other, U):
            res=U()
            res.scal=self.scal/other.scal
            for ii in range(6):
                res.vvec[ii]=self.vvec[ii]-other.vvec[ii]
            return res
            
    def disp(self):
        print('%f %s'%(self.scal,self.str()))
        
    def str(self):
        ss=''
        cs=['kPa','kg','mol','h','m','K']
        for ii in range(6):
            if self.vvec[ii] >0:
                if self.vvec[ii]==1:
                    ss=ss+cs[ii]+' '
                else:
                    ss=ss+cs[ii]+f"{self.vvec[ii]:.0f}"+' '
        for ii in range(6):
            if self.vvec[ii]<0:
                ss=ss+cs[ii]+f"{self.vvec[ii]:.0f}"+' '
        return '['+ss.strip()+']'
            

au=U('MPa h m-1')
bu=U('km h-1')
au.disp()
bu.disp()
cu=au*bu
cu.disp()
du=au/bu
du.disp()
        
            
            
                

1000.000000 [kPa h m-1]
1000.000000 [m h-1]
1000000.000000 [kPa]
1.000000 [kPa h2 m-2]


In [11]:
from pint import UnitRegistry

# Create a UnitRegistry instance
ureg = UnitRegistry()

# Define units from strings
unit_g_m2 = ureg('g m^-2')
unit_MPa_h = ureg('MPa h^-1')

# Combine units by multiplication and division
combined_unit_1 = unit_g_m2 * unit_MPa_h
combined_unit_2 = unit_g_m2 / ureg('m^2')

# Display the results
print(f"'g m^-2' multiplied by 'MPa h^-1' results in: {combined_unit_1}")
print(f"'g m^-2' divided by 'm^2' results in: {combined_unit_2}")

# Example with a value
quantity_1 = 10 * unit_g_m2
quantity_2 = 5 * unit_MPa_h
result_quantity = quantity_1 * quantity_2

print(f"\n10 g/m^2 multiplied by 5 MPa/h is: {result_quantity}")

# Access simplified or base units
print(f"The simplified result is: {result_quantity.to_base_units()}")

# Handle incompatible units (raises an error)
try:
    incompatible_addition = ureg('m') + ureg('s')
except Exception as e:
    print(f"\nCaught an expected error: {e}")


/opt/anaconda3/lib/python3.12/site-packages/dask/array/chunk_types.py:131: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.0)
  import scipy.sparse


'g m^-2' multiplied by 'MPa h^-1' results in: 1.0 gram * megapascal / hour / meter ** 2
'g m^-2' divided by 'm^2' results in: 1.0 gram / meter ** 4

10 g/m^2 multiplied by 5 MPa/h is: 50.0 gram * megapascal / hour / meter ** 2
The simplified result is: 13.88888888888889 kilogram ** 2 / meter ** 3 / second ** 3

Caught an expected error: Cannot convert from 'meter' ([length]) to 'second' ([time])
